In [1]:
import torch
import torch.nn as nn
from DataSource import test_dataloader
from UNet import U_Net
import numpy as np
from PIL import Image

model = U_Net(1,2)
model.load_state_dict(torch.load('model_weights_test.pth'))
model.eval() #模型评估
model = model.cuda()
def show_result(dataloader, model):
    count_of_img=1
    with torch.no_grad():
        for X, y in dataloader:
            X = X.cuda()
            pred = model(X)
            pred = pred.cpu()
            pred = nn.Softmax(dim=1)(pred)
            #pred = pred.int()
            pred = np.array(pred)
            #pred = np.where(pred>0.4,pred,0)
            pred = pred*255
            pred = pred.astype(np.int32)
            y = y.int()#直接转换会有问题，因为当初我们取label时为了使用交叉熵函数转成过Long型
            y = np.array(y)
            batch_size = pred.shape[0]
            for i in range(batch_size):
                tensor_this_time = pred[i][0]
                result_this_time = Image.fromarray(tensor_this_time)
                label_this_time = Image.fromarray(y[i]*255)
                result_this_time = result_this_time.convert("L")
                label_this_time = label_this_time.convert("L")
                result_this_time.save("result_"+str(count_of_img)+".jpg")
                label_this_time.save("label_"+str(count_of_img)+".jpg")
                count_of_img+=1

show_result(test_dataloader,model)

/home/ma-user/anaconda3/envs/PyTorch-1.8/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
